# Experimentos: Baseline com Rede Neural Clássica(MLP)

Este notebook estabelece baselines de performance com um modelo de Rede Neural Clássica(MLP) em datasets

# Dataset = Breast Cancer Wisconsin (Diagnostic)

In [1]:
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier  # Importando MLP
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

## Loading data and initial analysis

In [2]:
X, y = load_breast_cancer(return_X_y=True)  # Carregando o dataset

# Verificando as dimensões dos nossos dados
print(f"Dimensões dos dados: {X.shape}")


# Vendo a distribuição das classes (0 e 1)
print("\nDistribuição das classes:")
# Usando numpy para contar as ocorrências de cada classe
unique, counts = np.unique(y, return_counts=True)
for class_label, count in zip(unique, counts):
    print(f"Classe {class_label}: {count} ocorrências")


Dimensões dos dados: (569, 30)

Distribuição das classes:
Classe 0: 212 ocorrências
Classe 1: 357 ocorrências


## Split data into training and testing sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify -> mantém a proporção das classes no split

print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (426, 30)
Dimensões do conjunto de teste: (143, 30)


## Normalize data

In [4]:
# normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Ajusta o scaler e transforma os dados de treino
X_test = scaler.transform(X_test)  # Transforma os dados de teste com o mesmo scaler

print("\nDados normalizados")






Dados normalizados


## applying pca

In [5]:
from sklearn.decomposition import PCA

N_COMPONENTES_BC = 5
pca_bc = PCA(n_components=N_COMPONENTES_BC)


X_train_pca = pca_bc.fit_transform(X_train)
X_test_pca = pca_bc.transform(X_test)

print(f"PCA aplicado. Dimensões reduzidas para {N_COMPONENTES_BC} componentes.")

PCA aplicado. Dimensões reduzidas para 5 componentes.


# Training MLP model

In [6]:
# Treinando o modelo Rede neural MLP
mlp_breast = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_breast.fit(X_train_pca, y_train)  # Treinando o modelo com os dados de treino
print("\nMLP com PCA treinado com sucesso!")


MLP com PCA treinado com sucesso!


# Predictions

In [7]:
# predicting the test set com MLP
mlp_predictions_breast = mlp_breast.predict(X_test_pca)
# Avaliando o modelo
print("\nRelatório de Classificação PCA 5 features:") 
print(classification_report(y_test, mlp_predictions_breast, target_names=load_breast_cancer().target_names))


Relatório de Classificação PCA 5 features:
              precision    recall  f1-score   support

   malignant       0.93      0.96      0.94        53
      benign       0.98      0.96      0.97        90

    accuracy                           0.96       143
   macro avg       0.95      0.96      0.96       143
weighted avg       0.96      0.96      0.96       143



# Dataset = Student Performance

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

def run_mlp_pca(filepath, subject_name, n_components=5, separator=','):

    print(f" Iniciando processamento: MLP com PCA para conjunto de:({subject_name})    "))
    
    # Carregamento e Pré-processamento
    df = pd.read_csv(filepath, sep=separator)
    y_grades = df[['G1', 'G2', 'G3']]
    X_features = df.drop(columns=['G1', 'G2', 'G3'])
    y_final = np.where(y_grades['G3'] >= 10, 1, 0)
    X_final = pd.get_dummies(X_features, drop_first=True)

    # Divisão e Normalização
    X_train, X_test, y_train, y_test = train_test_split(
        X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
    )
    
    scaler = StandardScaler()
    X_train_scaled_full = scaler.fit_transform(X_train)
    X_test_scaled_full = scaler.transform(X_test)
    
    # Aplicação do PCA
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_scaled_full)
    X_test_pca = pca.transform(X_test_scaled_full)
    
    # SMOTE (nos dados já reduzidos com PCA)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pca, y_train)
    
    # Treinamento e Avaliação do MLP
    mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    mlp_model.fit(X_train_resampled, y_train_resampled)
    
    predictions = mlp_model.predict(X_test_pca)
    
    print(f"\n  Relatório de Classificação   ")
    print(classification_report(y_test, predictions, target_names=['Reprovado', 'Aprovado']))

In [9]:

path_portugues = '../data/student-por.csv'
path_matematica = '../data/student-mat.csv'


run_mlp_pca(filepath=path_portugues, subject_name="Português", separator=",")

print("\n" + "="*50 + "\n") 


run_mlp_pca(filepath=path_matematica, subject_name="Math", separator=";")

  Iniciando Experimento: MLP com PCA de 5 Componentes (Português)   

  Relatório de Classificação   
              precision    recall  f1-score   support

   Reprovado       0.22      0.28      0.25        25
    Aprovado       0.86      0.82      0.84       138

    accuracy                           0.74       163
   macro avg       0.54      0.55      0.54       163
weighted avg       0.76      0.74      0.75       163



  Iniciando Experimento: MLP com PCA de 5 Componentes (Math)   

  Relatório de Classificação   
              precision    recall  f1-score   support

   Reprovado       0.41      0.36      0.39        33
    Aprovado       0.70      0.74      0.72        66

    accuracy                           0.62        99
   macro avg       0.56      0.55      0.55        99
weighted avg       0.60      0.62      0.61        99

